In [ ]:
device = "cpu"
# device = "cuda"
import time,os,copy,torch,torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
import pickle
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(42)
np.random.seed(42)

# Plotting
import matplotlib.pyplot as plt

# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "16"

In [ ]:
n_qubits = 6                # Number of qubits
nqubits = n_qubits          
batch_size = 4              # Number of samples for each training step
q_depth = 1                 # Depth of the quantum circuit (number of variational layers)
q_delta = 0.01              # Initial spread of random quantum weights

tensor_length = n_qubits*(n_qubits-1)*q_depth+n_qubits*(q_depth-1)
print(tensor_length)

In [ ]:
def H_layer(nqubits):
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RZ_layer(w):
    for idx, element in enumerate(w):
        qml.RZ(element, wires=idx)

def entangling_layer(nqubits,weights):
    p = nqubits
    weights_ = (weight for weight in weights)
    for i in range(1,nqubits):
        for j in range(i):
            qml.CNOT(wires=[j,i])
            param = next(weights_)
            qml.RZ(param, wires=i)
            p+=1
            param = next(weights_)
            qml.RX(param, wires=i)
            p+=1
            qml.CNOT(wires=[j,i])

In [ ]:
dev = qml.device("default.qubit", wires=n_qubits)
@qml.qnode(dev, interface="torch")

def quantum_net(q_input_features, q_weights_flat):
    q_weights = q_weights_flat
    H_layer(n_qubits)
    RZ_layer(q_input_features)
    entangling_layer(nqubits,q_weights)
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)
    
class DressedQuantumNet(nn.Module):

    def __init__(self):

        super().__init__()
        self.pre_net = nn.Linear(128, n_qubits)
        self.q_params = nn.Parameter(q_delta * torch.randn(tensor_length))
        self.post_net = nn.Linear(n_qubits, 5)

    def forward(self, input_features):

        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        return self.post_net(q_out)

In [ ]:
class DressedClassicalNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.pre_net = nn.Linear(128, 6)
        self.mid_net = nn.Linear(6,6)
        self.post_net = nn.Linear(6, 5)

    def forward(self, input_features):

        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0
        mid_out = self.mid_net(q_in)
        return self.post_net(mid_out)

In [ ]:
# model_hybrid = copy.deepcopy(torch.nn.Sequential(*(list(spikeaware_model.children())[:-1])))
# for param in model_hybrid.parameters():
#     param.requires_grad = False
# # Notice that model_hybrid.fc is the last layer of ResNet18
# model_hybrid.fc = DressedQuantumNet()

# # Use CUDA or CPU according to the "device" object.
# model_hybrid = model_hybrid.to(device)
# display(model_hybrid)

In [ ]:
class_names = [
    "6",
    "7",
    "8",
    "9",
]
num_classes = len(class_names)

import idx2numpy 
import numpy as np 
arr3 = []
arr4 = []
file1 = './MNIST/TorchvisionDatasetWrapper/raw/train-images-idx3-ubyte' 
# file1 = './HybridQSNN/EMNIST/raw/gzip/emnist-digits-train-images-idx3-ubyte'
arr = idx2numpy.convert_from_file(file1)
# file2 = './HybridQSNN/EMNIST/raw/gzip/emnist-digits-train-labels-idx1-ubyte' 
file2 = './MNIST/TorchvisionDatasetWrapper/raw/train-labels-idx1-ubyte' 
arr2 = idx2numpy.convert_from_file(file2)
for i in range(len(arr)):
    if arr2[i] in [6,7,8,9]:
        arr3.append(arr[i])
        arr4.append(arr2[i])

In [ ]:
n_steps = 10
train_images = np.asarray(arr3,dtype=np.float32) / 255.0
train_labels = np.asarray(arr4,dtype=np.int64)
train_labels -= 6
display(train_labels)

These are the 4 classes that we perform the classification on.

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.axis("off")
    plt.title(class_names[train_labels[i]])

In [ ]:
# test_sequences = train_sequences[:int(len(train_sequences)*0.3)]
# test_labels = train_labels[:int(len(train_labels)*0.3)]
# train_sequences = train_sequences[int(len(train_sequences)*0.7):]
# train_labels = train_labels[int(len(train_labels)*0.7):]
split = np.array_split(train_images,[int(len(train_images)*0.7)])
train_sequences = split[0]
test_sequences = split[1]
print(len(split[0]),len(split[1]))
split = np.array_split(train_labels,[int(len(train_labels)*0.7)])
train_labels = split[0]
test_labels = split[1]
print(len(split[0]),len(split[1]))
datalength = 5000

train_sequences = np.tile(train_sequences[:, None], (1, n_steps, 1, 1))[:datalength]
test_sequences = np.tile(test_sequences[:, None], (1, n_steps, 1, 1))
train_labels = train_labels[:datalength]
test_labels = test_labels

print(len(test_labels),len(test_sequences),len(train_labels),len(train_sequences))

In [ ]:
model_hybrid=torch.load("./model_saves/4_class_KMNIST/6x2_zz_24_0.9641426205635071_0.1356203258037567.h5")
display(model_hybrid)

Here we try to add salt & pepper noise to the test_sequences and then we test our model on them.

In [ ]:
import random
def salt_pepper(test_sequences,test_labels,prob,salt=0,pepper=1):
  sampled_images = copy.deepcopy(test_sequences)
  sampled_labels = copy.deepcopy(test_labels)
  for j in range(len(test_labels)):
    for i in range(np.shape(sampled_images[j])[0]):
      for y in range(28):
        for x in range(28):
          r = random.random() 
          if r < prob:
            sampled_images[j][i][y][x] = salt
          elif r > 1-prob:
            sampled_images[j][i][y][x] = pepper
  return sampled_images,sampled_labels

In [ ]:
def test_noise(input_model,prob,salt=0,pepper=1):
    test_x , tst_labels = salt_pepper(test_sequences,test_labels,prob=prob,salt=0,pepper=1)
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels

In [ ]:
test_noise(model_hybrid,prob=0.01,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.02,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.03,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.04,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.05,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.06,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.07,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.08,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.09,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.1,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.2,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.3,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.3,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.4,salt=0,pepper=1)

In [ ]:
test_noise(model_hybrid,prob=0.5,salt=0,pepper=1)

--------x-------------x------------x------------x-------------x-------------x--------------------x--------------x--------x-------------x------------x------------x-------------x-------------x--------------------x

In the next section we add gaussian noise to the images:

In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.2**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)

test_noise(model_hybrid)

In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.2**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)


In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.3**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)


In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.4**0.5
mean = 0

for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)

def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)

In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.5**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)


In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.6**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)


In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.7**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)

In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.8**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)


In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 0.9**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)


In [ ]:
sampled_images = copy.deepcopy(test_sequences)
sampled_labels = copy.deepcopy(test_labels)
sigma = 1.0**0.5
mean = 0
for j in range(len(test_labels)):
  for _ in range(len(sampled_images[j])):
    im = sampled_images[j][_] 
    row,col = sampled_images[j][_].shape
    # gauss = np.array(np.random.normal(mean,sigma,size=(28,28))).reshape(row,col)
    gauss = np.array(np.random.randn(row*col)) #* var ** 0.5 + mean
    gauss = gauss*sigma + mean
    gauss = gauss.reshape(row,col)
    sampled_images[j][_] = sampled_images[j][_] + gauss
r=np.random.randint(0,9000)
plt.title(str(class_names[test_labels[r]]))
plt.imshow(sampled_images[r][0],cmap=plt.cm.binary)
def test_noise(input_model):
    # test_x , tst_labels = gaussian_noise(test_sequences,test_labels,mean =mean,var = var)
    test_x , tst_labels =sampled_images,test_labels
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = tst_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())
    print('here is a random image from the generated noisy data:')
    r=np.random.randint(0,9000)
    plt.title(str(class_names[tst_labels[r]]))
    plt.imshow(test_x[r][0],cmap=plt.cm.binary)

    # return test_x,tst_labels
test_noise(model_hybrid)
